In [11]:
import csv
import os
import tkinter as tk
from tkinter import messagebox, simpledialog, Toplevel, Text, Scrollbar, VERTICAL, RIGHT, Y, END
from datetime import datetime, timedelta

BOOK_FILE = "Books.csv"

if not os.path.exists(BOOK_FILE):
    with open(BOOK_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Book_ID", "Title", "Author", "Year",
            "Genre", "Available", "Borrower", "Return_Date"
        ])

class BookNode:
    def __init__(self, book):
        self.book = book
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def insert(self, book):
        node = BookNode(book)
        node.next = self.head
        self.head = node

    def delete(self, book_id):
        current = self.head
        prev = None
        while current:
            if current.book["Book_ID"] == book_id:
                if prev:
                    prev.next = current.next
                else:
                    self.head = current.next
                return True
            prev = current
            current = current.next
        return False

    def to_list(self):
        books = []
        current = self.head
        while current:
            books.append(current.book)
            current = current.next
        return books

class LibrarySystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Library Book Inventory System")
        self.root.attributes("-fullscreen", True)

        self.book_list = LinkedList()
        self.book_table = {}

        self.load_books()
        self.main_menu()

    def load_books(self):
        with open(BOOK_FILE, newline="") as f:
            reader = csv.DictReader(f)
            for row in reader:
                row["Book_ID"] = row["Book_ID"].strip().upper()

                self.book_list.insert(row)
                self.book_table[row["Book_ID"]] = row

    def save_books(self):
        with open(BOOK_FILE, "w", newline="") as f:
            fieldnames = [
                "Book_ID", "Title", "Author", "Year",
                "Genre", "Available", "Borrower", "Return_Date"
            ]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for book in self.book_list.to_list():
                writer.writerow(book)

    def main_menu(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        tk.Label(
            self.root,
            text="Library Book Inventory System",
            font=("Arial", 28)
        ).pack(pady=20)

        tk.Button(self.root, text="View Available Books", width=30, command=self.view_books).pack(pady=10)
        tk.Button(self.root, text="Borrow Book", width=30, command=self.borrow_book).pack(pady=10)
        tk.Button(self.root, text="Return Book", width=30, command=self.return_book).pack(pady=10)
        tk.Button(self.root, text="Exit", width=30, command=self.exit_system).pack(pady=10)

    def view_books(self):
        display = ""
        for book in self.book_list.to_list():
            if book["Available"].lower() == "yes":
                display += (
                    f"{book['Book_ID']} | "
                    f"{book['Title']} | "
                    f"{book['Author']} | "
                    f"{book['Year']} | "
                    f"{book['Genre']}\n"
                )

        if not display:
            display = "No available books."

        window = Toplevel(self.root)
        window.attributes("-fullscreen", True)

        scrollbar = Scrollbar(window, orient=VERTICAL)
        text = Text(window, yscrollcommand=scrollbar.set)
        scrollbar.config(command=text.yview)

        scrollbar.pack(side=RIGHT, fill=Y)
        text.pack(expand=True, fill="both")

        text.insert(END, display)
        text.config(state="disabled")

        tk.Button(window, text="Back", command=window.destroy).pack(pady=10)

    def borrow_book(self):
        book_id = simpledialog.askstring("Borrow Book", "Enter Book ID:")
        if not book_id:
            return

        book_id = book_id.strip().upper()

        if book_id not in self.book_table:
            messagebox.showerror("Error", "Invalid Book ID.")
            return

        book = self.book_table[book_id]

        if book["Available"] == "No":
            messagebox.showerror("Error", "Book is not available.")
            return

        borrower = simpledialog.askstring("Borrower", "Enter borrower name:")
        if not borrower:
            return

        book["Available"] = "No"
        book["Borrower"] = borrower
        book["Return_Date"] = (datetime.now() + timedelta(days=21)).strftime("%Y-%m-%d")

        self.save_books()
        messagebox.showinfo("Success", "Book borrowed successfully.")

    def return_book(self):
        book_id = simpledialog.askstring("Return Book", "Enter Book ID:")
        if not book_id:
            return

        book_id = book_id.strip().upper()

        if book_id not in self.book_table:
            messagebox.showerror("Error", "Invalid Book ID.")
            return

        book = self.book_table[book_id]

        if book["Available"] == "Yes":
            messagebox.showinfo("Info", "This book is not currently borrowed.")
            return

        due_date = datetime.strptime(book["Return_Date"], "%Y-%m-%d")
        overdue_days = (datetime.now() - due_date).days
        fine = max(0, overdue_days * 0.2)

        book["Available"] = "Yes"
        book["Borrower"] = ""
        book["Return_Date"] = ""

        self.save_books()

        if fine > 0:
            messagebox.showinfo("Returned", f"Book returned.\nFine: RM {fine:.2f}")
        else:
            messagebox.showinfo("Returned", "Book returned successfully. No fine.")

    def exit_system(self):
        self.save_books()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    LibrarySystem(root)
    root.mainloop()
